# TODO fase 1
 - calculate predictions for a stock fx apple
 - statistics for the predictions: distribution of errors, percentage prediction with the right direction:  close>open, open<close
 - visualize ohlc as bands and on stock curve
 - split in train, test so that test data are taken from time periodens after the training data and possibly from stocks that have not been used for training 

 - tjek quantile objective function
 - objectiv function that weigh open and closing prices higher

 - add day of week to training data
                                                                                                                 
                                                                                                                 
# TODO fase 2
 - language modelling

# DONE 
 - speed up training: improve the pandas part by 30 to 60 times


In [ ]:
#default_exp finance.data

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export
from lib.data.lists import *
from pathlib import Path

import pandas as pd
import numpy as np

#import matplotlib.pyplot as plt

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [ ]:
# Version of the SimFin Python API.
sf.__version__

# Define location of Simfin data and license

In [ ]:
# export
def initiateSimFin(key='free'):
    # SimFin data-directory.
    sf.set_data_dir('~/simfin_data/')
    # SimFin load API key or use free data.
    sf.load_api_key(path='~/simfin_api_key.txt', default_key=key)
    return Path.home()/"simfin_data"

# Prepare data for prediction of next days stock prices the next day
Create OHLC price changes as percentages relative to the close of the previous day. Furthermore the ohlc is supplemented by the previous days closing.

In [ ]:
# export

PREV_CLOSE = "previous_close"
def alignPrices(df_prices):
    # Make a new column to show the closing price from the previous day on the same line as current day. 
    # This is done using the shift function for the dataserie
    # 
    # Result: All stock and prices are listed in the same tabel. Therefore, the firat priceline of each stock vil now 
    # contain the close of the previous stock. For the first stock this value vil be "nan". 
    # These incoherent pricelines are removed in the nest step
    
    #if PREV_CLOSE not in df_prices.columns: 
    if not PREV_CLOSE in df_prices.columns: 
        df_prices.insert(df_prices.columns.get_loc(OPEN), PREV_CLOSE, df_prices[CLOSE].shift(), allow_duplicates=False)

    #identify the first stock in each stockgroup and the remove it
    stock_name    = df_prices.index.get_level_values(0)
    new_stock     = np.ones(len(df_prices), dtype=bool)
    new_stock[1:] = stock_name[0:len(stock_name)-1] != stock_name[1:len(stock_name)]
    df_prices.drop(df_prices.index[new_stock], axis=0, inplace=True)
    return df_prices

#Convert OHLC to percentages of the previous days closing price
def convertToPercentages(df_prices):
    df_prices[[OPEN,CLOSE,LOW,HIGH]] = df_prices[[OPEN,CLOSE,LOW,HIGH]].sub(df_prices[PREV_CLOSE],axis=0)\
                                        .div(df_prices[PREV_CLOSE],axis=0)
    return df_prices

In [ ]:
# export

#   extrem_increase,extrem__decrease = [], 2.5, -.73
#   extrem_increase,extrem__decrease = [], 1.5, -.5
EXTREM_HL = "extrem_hl"
def flagstocks_extrem_hl(df_prices, extrem_increase, extrem__decrease):
    #remove mature stocks with "nan" prices
    if EXTREM_HL in df_prices.columns: 
        df_prices.drop(labels=EXTREM_HL,axis=1,inplace=True)
                       
    gt_p = df_prices[[HIGH]].gt(extrem_increase)
    gt_p.insert(0,"lt",df_prices[[LOW]].lt(extrem__decrease))
    df_prices.insert(df_prices.columns.get_loc(SIMFIN_ID)+1,EXTREM_HL,gt_p.any(axis=1))    
    return df_prices

TOO_FEW_DAYS = "too_few_days"
def flagstocks_too_few_trading_days(df_prices, minimum_tradingdays):
    grp_res = df_prices.groupby(["Ticker"]).apply(lambda group: len(group) < minimum_tradingdays) 
    grp_res.name = TOO_FEW_DAYS
    merged = df_prices[[SIMFIN_ID]].join(grp_res, on=TICKER)
    if TOO_FEW_DAYS in df_prices.columns: 
        df_prices.drop(labels=TOO_FEW_DAYS,axis=1,inplace=True)
    df_prices.insert(df_prices.columns.get_loc(SIMFIN_ID)+1, TOO_FEW_DAYS, merged[TOO_FEW_DAYS].values)
    return df_prices

HAS_NAN_DAYS = "has_nan_days"
def flagstocks_with_nan_days(df_prices):
    grp_res = df_prices.groupby(["Ticker"]).apply(lambda group: group[[OPEN,LOW,HIGH,CLOSE]].isnull().values.any() ) 
    grp_res.name = HAS_NAN_DAYS
    merged = df_prices[[SIMFIN_ID]].join(grp_res, on=TICKER)
    if HAS_NAN_DAYS in df_prices.columns: 
        df_prices.drop(labels=HAS_NAN_DAYS,axis=1,inplace=True)
    df_prices.insert(df_prices.columns.get_loc(SIMFIN_ID)+1, HAS_NAN_DAYS, merged[HAS_NAN_DAYS].values)
    return df_prices

def findInValidStocks(df_prices):
    flags  = [HAS_NAN_DAYS,TOO_FEW_DAYS,EXTREM_HL]
    groups = df_prices.groupby(["Ticker"])
    return np.array([name for (name,group) in groups if group[flags].any(axis=1).any()])

def findValidStocks(df_prices):
    flags  = [HAS_NAN_DAYS,TOO_FEW_DAYS,EXTREM_HL]
    groups = df_prices.groupby(["Ticker"])
    return np.array([name for (name,group) in groups if not group[flags].any(axis=1).any()])


# find valid and invalid stocks
#df_prices[df_prices[[HAS_NAN_DAYS,TOO_FEW_DAYS,EXTREM_HL]].any(axis=1)]
len(findValidStocks(df_prices)), len(findInValidStocks(df_prices))

# Create dataset
The dataset must a batch with number of stock = batch_size  
Each sequence of stockprice (ohlc) return from the dataset must be of the samme sequence_length
During the training the network will process the sequence day by day

At the beginning of each epoch
    It must be possible to shuffle the stock 
    It must be possible to shuffle the start date of the stockprices for each stock 

The dataframe will remain fixed during the training using indirect indexing 
This will be faster and use less memory for large dataframes.


In [ ]:
#test of logic to interleave data from df in np_prices
import numpy as np
import pandas as pd

open_=np.arange(0,12,4)
high =np.arange(1,12,4)
low  =np.arange(2,12,4)
close=np.arange(3,12,4)

df = pd.DataFrame({"open":open_,"high":high,"low":low,"close":close})
columns=df.columns

n_columns=len(columns)
len_array=df.shape[0]
np_prices = np.empty(len_array*n_columns)

for (i,c) in enumerate(columns):
    np_prices[i::n_columns] = df[c].values
index = 2
seq_length = 1
index*n_columns, (seq_length+index)*n_columns, np_prices

(8, 12, array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]))

In [ ]:
# export
from lib.data.lists import *
import torch
torch.set_default_dtype(torch.float64)

#Create a dataset that uses all the stocks and daily prices 
#In order to save memory the df_prices is the same for both training and validation.
#np_prices contains interleaved prices of the columns used for training/validation
#fx if the require columns are "open", "close". Then np_prices contain row, columns values from 
#df_price in the following order : open_row1, close_row1, open_row2, close_row2, ...

#for now self.np_prices are dublicated. Thus using more memory than needed.
#Test and validation contain different stock taken drom the same df_prices

#stocks    : are the stock tickers
#stock_days: is the number of days pr stock. The number of days is in different for each stock
#            The total number of samples is stock_days.sum()
#batch_size: is the number of stocks in a batch
#seq_length: is the number of stocks days in a batch. 
#the number of sample in the batch is seq_length*len(column_names)

#As for now the dataset progress sequentially through the data during an epoch from a random offset. 
#Notice that there is no end of sequence-token to reset the model when the sequencen wraps around

class OHLCDataset(torch.utils.data.Dataset):
    #x, y significes input vs output
    def __init__(self, df_prices, stocks, column_names, seq_length): 
        self.df_prices    = df_prices
        self.stock_grps   = self.df_prices.groupby(["Ticker"])
        self.stocks       = np.array(list(self.stock_grps.groups)) if stocks is None else np.array(stocks)
        self.seq_length   = seq_length
        self.column_names = column_names
        self.ix_columns   = [self.df_prices.columns.get_loc(k) for k in self.column_names]

        #Index to all start of sequences in the dataset
        #The target is taken from the day after the sequence
        self.idx_seq      = None
        self.prices       = None

    def __len__(self): 
        if self.idx_seq is None: self.initializeSquenzing()
        return len(self.idx_seq)

    def __getitem__(self, index):
        #pt: price target
        #pi: price input
        ix = self.idx_seq[index,:]
        
        #pi = torch.tensor(self.price_values[ix[0]:ix[1]])
        #pt = torch.tensor(self.price_values[ix[1]])
        pi = torch.tensor( self.np_prices[ix[1]:ix[0]] )
        pt = torch.tensor( self.np_prices[ix[0]:ix[0]+len(self.column_names)] )
        return pi, pt #, index
    
    """
    def __getitem__(self, index):
        #pt: price target
        #pi: price input
        ix = self.idx_seq[index,:]
        #pi = torch.tensor(self.price_values[ix[0]:ix[1]])
        #pt = torch.tensor(self.price_values[ix[1]])
        pi = torch.tensor(self.price_values[ix[1]:ix[0]])
        pt = torch.tensor(self.price_values[ix[0]])
        return pi, pt #, index
    """

    #stocks: stocks that you want the price_input and target for    
    def getBatch(self, stocks):
        #pt: price target
        #pi: price input
        #extract the rows in df_proces containing the stocks in the batch
        idx = self.getSequenceIndexing(stocks);
        
        
        """
        df_prices_selected = self.df_prices[ self.df_prices.index.get_level_values(0).isin(stocks) ]
        stock_grps         = df_prices_selected.groupby(["Ticker"])
        idx, seq_len = [], self.seq_length
        for name,group in stock_grps:
            idx.extend(group["idx"].values[seq_len:len(group)])     
        idx = np.array(idx,dtype=int)
    
        pt = self.df_prices.iloc[idx]
        pi = [self.df_prices.iloc[ib:ie, self.ix_columns].values for ib,ie in zip(pt.idx - seq_len, pt.idx)]
        return np.array(pi), self.df_prices.iloc[idx].copy()
        """
        
    def initializeSquenzing(self):
        #Notice that the dataset cannot be used for a RNN where the data should been returned from a sliding window

        #Calculate the start and end of all sequences so that the dataloader can load and shuffle efficiently.  
        #add an index column to df_prices that we can used to calcualte start and end of sequences
        if not "idx" in self.df_prices.columns:
            self.df_prices.insert(loc=0, column="idx", value=np.arange(len(self.df_prices),dtype=int))
    
        #create one array "prioces with interleaved values from training.columns iinterleaved in 
        #fx if the require columns are "open", "close". Then prices contain row, columns values from 
        #df_price in the following order : open_row1, close_row1, open_row2, close_row2, ...
        self.np_prices = np.empty(self.df_prices.shape[0]*len(self.column_names))
        for (i,c) in enumerate(self.column_names):
            self.np_prices[i::len(self.column_names)] = self.df_prices[c].values

        self.ix_columns = [self.df_prices.columns.get_loc(k) for k in self.column_names]
        self.idx_seq    = self.getSequenceIndexing(self.stocks)
        
    def getSequenceIndexing(self,stocks):
        #get an index to all valid samples in self.prices
        idx_begin = [] #list of valid dataset index into the dataframe        
        for stock in stocks:
            group = self.stock_grps.get_group(stock)
            idx_begin.extend( group["idx"].values[self.seq_length:len(group)] )        
        idx_begin    = np.array(idx_begin,dtype=int)    
        idx_seq      = np.empty( (len(idx_begin),2), dtype=int)  
        #idx_seq[:,0] = (idx_begin+self.seq_length)*len(self.column_names)
        idx_seq[:,0] = idx_begin*len(self.column_names)
        idx_seq[:,1] = idx_seq[:,0] - self.seq_length*len(self.column_names)
        return idx_seq
        """    
        #get an index to all valid samples
        idx_begin = [] #list of valid dataset index into the dataframe        
        for stock in self.stocks:
            group = self.stock_grps.get_group(stock)
            #idx_begin.extend( group["idx"].values[:len(group)-self.seq_length] )        
            idx_begin.extend( group["idx"].values[self.seq_length:len(group)] )        
        self.idx_seq = np.empty( (len(idx_begin),2), dtype=int)  
        self.idx_seq[:,0] = idx_begin
        self.idx_seq[:,1] = self.idx_seq[:,0]-self.seq_length
        """
    def dataloader(self, batch_size:int, shuffle:bool, num_workers:int=0, drop_last=False):
        self.initializeSquenzing()
        return torch.utils.data.DataLoader(self, batch_size=batch_size, shuffle=shuffle,
                                           num_workers=num_workers, drop_last=drop_last)
    
    def split2train_test(self, test_percentage):
        #split the stocks in train and test stocks
        #It might be better to make a split in historical vs new prices 
        ix_all   = np.arange(len(self.stocks),dtype=int)
        np.random.shuffle(ix_all)
        
        nb_test  = int(round(test_percentage*len(self.stocks)))
        ix_train = ix_all[nb_test:]
        ix_test  = ix_all[:nb_test]
        return self.__class__(self.df_prices, self.stocks[ix_train], self.column_names, self.seq_length),\
               self.__class__(self.df_prices, self.stocks[ix_test],  self.column_names, self.seq_length)


In [ ]:
# export 

#extrem_increase,extrem__decrease = 2.5, -.73
#extrem_increase,extrem__decrease = 1.5, -.5
def prepareSimFinData(minimum_tradingdays=180, extrem_increase = 1.5, extrem__decrease = -0.5):

    #Load shareprices for testing and development
    dataPath = initiateSimFin(key='free')
    print(f"dataPath:{dataPath} exists:{dataPath.exists()}")

    # Data for USA.
    market = 'us'
    # Daily Share-Prices.
    df_prices = sf.load_shareprices(variant='daily', market=market)
    df_prices.head()

    # load and identify valid data
    alignPrices(df_prices)
    convertToPercentages(df_prices)

    flagstocks_extrem_hl(df_prices, extrem_increase, extrem__decrease)
    flagstocks_too_few_trading_days(df_prices,minimum_tradingdays)
    flagstocks_with_nan_days(df_prices)

    validStocks, inValidStocks = findValidStocks(df_prices), findInValidStocks(df_prices)

    stock_grps = df_prices.groupby(["Ticker"])
    stocks     = np.array(list(stock_grps.groups))
    sizes      = stock_grps.size()
    print(f"number of stocks:         {len(stocks)}")
    print(f"number of valid stocks:   {len(validStocks)}")
    print(f"number of invalid stocks: {len(inValidStocks)}")

    print(f"smallest pricelines pr stock: {sizes.sort_values()[:5]}")
    print(f"longest pricelines pr stock:  {sizes.sort_values()[-5:]}")
     
    return df_prices, stocks, validStocks, inValidStocks

predict_prefix="predict_"
[PREDICT_OPEN,PREDICT_HIGH,PREDICT_LOW,PREDICT_CLOSE] = [predict_prefix+OPEN,\
                                                         predict_prefix+HIGH,\
                                                         predict_prefix+LOW,\
                                                         predict_prefix+CLOSE]
def predict_stocks(dataset, modelmanager, stocks, tfm_input ):
    price_sequences, price_targets = dataset.getBatch(stocks=stocks)
    predictions    = modelmanager.predict(price_sequences, tfm_input)
    
    # inser the preduction in price_targets
    prediction_columns = [ predict_prefix + name for name in dataset.column_names ]
    print(prediction_columns)
    for idx,name in enumerate(prediction_columns):
        print(idx, name)
        if name in price_targets.columns : 
            price_targets.drop([name], axis='columns', inplace=True)
        else:    
            price_targets.insert(len(price_targets.columns), name, predictions[:,idx-1].numpy())
    return price_targets, prediction_columns

In [ ]:
#normalization_faktor = 2.697750e-02
#TODO add normalized datacolumn to df_prices indstead of replacing them
normalized_prefix="normalized_"
[NORM_OPEN,NORM_HIGH,NORM_LOW,NORM_CLOSE] = [normalized_prefix+OPEN,\
                                             normalized_prefix+HIGH,\
                                             normalized_prefix+LOW,\
                                             normalized_prefix+CLOSE]

def normalizeData( df_prices, columns, normalization_faktor=2.7e-02, normalized_prefix=normalized_prefix):
    #normalize the training data
    if not normalization_faktor==1.0 :
        normalized_columns = ["normalized_"+ name for name in columns]
        df_prices[normalized_columns] = df_prices[columns].div(normalization_faktor,axis="columns").values
    return df_prices,normalized_columns


# Test


In [ ]:
df_prices, stocks, validStocks, inValidStocks = prepareSimFinData()

In [ ]:
#statistics on the mature stocks
df_prices.loc[validStocks,[PREV_CLOSE,OPEN,HIGH,LOW,CLOSE]].describe(percentiles=[0.0002, 0.25, 0.75, 0.9998])        

In [ ]:
%%time 

from lib.learner.learner import*
from lib.learner.optimizers import*
from lib.model.model import*
from lib.model.modelmanager import*
import torch.nn as nn

seq_length = 32
training_column_names = [CLOSE]#[OPEN,HIGH,LOW,CLOSE] #[OPEN,#CLOSE]

df_prices, training_column_names = normalizeData(df_prices,training_column_names)

#each priceline has 4 datapoint:[OPEN,LOW,HIGH,CLOSE]
#batchsize is the number of stocks processed in parallel

#ohlc: almost all day to day variation are in the the range -1 to 1.
#For now we do not normalize the input. However we will have to do it sooner or later

#[PREDICT_OPEN,PREDICT_HIGH,PREDICT_LOW,PREDICT_CLOSE] = ["predict_"+OPEN,"predict_"+HIGH,"predict_"+LOW,"predict_"+CLOSE]

#self.column_names = [OPEN,HIGH,LOW,CLOSE], [OPEN,CLOSE]
ohlc_ds = OHLCDataset(df_prices, stocks=validStocks, column_names=training_column_names, seq_length=seq_length)
ohlc_ds.initializeSquenzing()
print(f"training columns = {training_column_names}")

train_ds, test_ds = ohlc_ds.split2train_test(0.25)
print(f"number of stocks, train stocks, test stocks: {len(ohlc_ds.stocks)}, {len(train_ds.stocks)}, {len(test_ds.stocks)}")

databunch = DataBunch(train_ds.dataloader(batch_size=2048, shuffle=True,  drop_last=True), \
                      test_ds.dataloader( batch_size=4096, shuffle=False, drop_last=False), \
                      c_in=len(ohlc_ds.column_names), c_out=len(ohlc_ds.column_names))

#batch = next(iter(databunch.train_dl))
#batch[0].dtype, batch[0].shape, view(batch[0]).shape

%time l = [len(b[0]) for b in databunch.train_dl]

#print("the following lengths must be the same")
#%time stock_days = [len(ohlc_ds.stock_grps.get_group(stock)) for stock in ohlc_ds.stocks]
#print(len(ohlc_ds), sum(stock_days)-len(stock_days)*(seq_length+1))

In [ ]:
#import scipy.special
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from lib.finance.graphs import *

data_column = training_column_names[0]
stocks      = ohlc_ds.stocks
df_select   = df_prices.loc[stocks,data_column]
measured    = df_select.values
x_min,x_max = -measured.min(), measured.max()
x           = np.linspace(x_min,x_max, 100)
hist, edges = np.histogram(measured, density=True, bins=100)

# Normal Distribution
#x = np.linspace(x_min,x_max, 1000)
#pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
#cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1 = make_plot("Histogram for "+data_column, hist, edges, x)
show(gridplot([p1], ncols=1, plot_width=1000, plot_height=400))  


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_data.external.ipynb.
Converted 02_lists.ipynb.
Converted 03_images.ipynb.
Converted 05_Learner.ipynb.
Converted 05_model.ipynb.
Converted 06_modelmanger.ipynb.
Converted 07_optimizers.ipynb.
Converted app_image_01_mnist_optimizers.ipynb.
Converted app_image_02_imagenette_optimizers.ipynb.
Converted fin_01_candlestick.ipynb.
Converted fin_02_simfin_data.ipynb.
Converted fin_02_simfin_generated_data.ipynb.
Converted fin_02_simfin_training.ipynb.
Converted fin_03_graphs.ipynb.
Converted index.ipynb.
